In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from config import gmap_api_key

In [8]:
mental_df = pd.read_csv("Resources/mental.csv")
anxiety_depression = mental_df.loc[(mental_df["Group"] == "By State") \
                                     &(mental_df["Indicator"] == \
                                       "Symptoms of Anxiety Disorder or Depressive Disorder"),:]
anxiety_depression_states = pd.DataFrame({"State":anxiety_depression["State"].unique(),"Lat":"","Lng":""})
for index,row in anxiety_depression_states.iterrows():
    state = row["State"]
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={state}&key={gmap_api_key}"
    response = requests.get(target_url).json()
    try:
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']
        anxiety_depression_states.loc[index,"Lat"]=lat
        anxiety_depression_states.loc[index,"Lng"]=lng
    except:
        print(f"{state} not found. Skipping...")
        pass
state_anxiety_depression = pd.merge(anxiety_depression,anxiety_depression_states, how="left",on="State")
state_anxiety_depression
state_anxiety_depression["Time Period Start Date"]=pd.to_datetime(state_anxiety_depression["Time Period Start Date"])
state_anxiety_depression["Time Period End Date"]=pd.to_datetime(state_anxiety_depression["Time Period End Date"])

In [9]:
vote_count = pd.read_csv("Resources/2020_vote.csv")
hpsa_df = pd.read_csv("Resources/HPSA.csv")
state_dataset = pd.merge(state_anxiety_depression,vote_count,on="State")
state_dataset=pd.merge(state_dataset,hpsa_df,on="State")
state_dataset.to_csv("Resources/state_dataset.csv",index=False)